# Compare pilot annotations
In which we compare the annotations (over sample posts) provided in the pilot study to the gold standard annotations.

In [7]:
import pandas as pd
import re

## Load data

In [38]:
label_matcher = re.compile('(\[\[[^\]]+\]\])(\{[A-Z]+\})(\{[^\}]+\})')
def extract_labels(x):
    x_groups = label_matcher.findall(x)
    # we want the name and the ID, lat, lon if available
    x_data = map(lambda y: [y[0].replace('[[','').replace(']]',''), 
                            y[2].replace('{','').replace('}','').split(',')], x_groups)
    x_data_formatted = []
    for x_d in x_data:
        x_name = x_d[0]
        if(len(x_d[1]) < 3):
            x_id = x_d[1][0]
            if(x_id != 'UN' and x_id != 'VAGUE'):
                x_id = int(x_id)
            x_lat = 0
            x_lon = 0
        elif(len(x_d[1]) >= 3):
            x_id = int(x_d[1][0])
            x_lat = float(x_d[1][1])
            x_lon = float(x_d[1][2])
        x_data_formatted.append([x_name, x_id, x_lat, x_lon])
    return x_data_formatted
sample_annotated_posts = list(open('../../data/facebook-maria/all_group_sample_statuses_annotated_topo.txt'))
test_labels = extract_labels(sample_annotated_posts[0])
print(test_labels)
# print('\n'.join(sample_annotated_posts))

[['ingenio', 5184195284, 18.0834124, -65.8729305], ['yabucoa', 1944827110, 18.0504228, -65.8793678], ['humacao', 1945948351, 18.1501953, -65.826731], ['candeloro arriba', 4468392, 18.10191, -65.83683]]


In [40]:
sample_label_list = map(extract_labels, sample_annotated_posts)
for sample_labels in sample_label_list:
    print(sample_labels)

[['ingenio', 5184195284, 18.0834124, -65.8729305], ['yabucoa', 1944827110, 18.0504228, -65.8793678], ['humacao', 1945948351, 18.1501953, -65.826731], ['candeloro arriba', 4468392, 18.10191, -65.83683]]
[['Quintas De Guasimas', 'UN', 0, 0], ['Bda Marin', 5184347328, 0, 0]]
[['carr 104', 22196127, 18.2238, -67.1563], ['Mayaquez', 4467818, 0, 0]]
[['San Crist\xc3\xb3bal Apartments', 'UN', 0, 0], ['Tres Caminos', 5184329391, 18.1935967, -66.2910767], ['Tres Camino', 5184329391, 18.1935967, -66.2910767], ['Barrancas', 4468210, 0, 0], ['la gallera', 5184329405, 18.1468483, -66.3015871]]
[['San Juan', 4468264, 0, 0], ['Carolina', 4468017, 0, 0], ['Bayam\xc3\xb3n', 4468234, 0, 0], ['Camuy', 4467841, 0, 0], ['Aguada', 4468470, 0, 0], ['Mayag\xc3\xbcez', 4467818, 0, 0], ['A\xc3\xb1asco', 4468537, 0, 0], ['Carolina', 4468017, 0, 0], ['aeropuerto Rafael Hern\xc3\xa1ndez', 4567576, 18.4955, -67.13493], ['Aguadilla', 4468292, 0, 0], ['Antigua Base Ramey', 506154128, 0, 0], ['Aguadilla', 4468292, 0, 

## Load data from sheet
Now we can compare these annotations to the annotations collected from the first annotator.

Failed attempts at accessing Google Sheets from the notebook:

In [55]:
# code from 
from apiclient import discovery
from httplib2 import Http
from oauth2client import file, client, tools
SCOPES = 'https://www.googleapis.com/auth/drive.metadata.readonly'
client_secret_file = '../../data/GoogleSheets/client_secret.json'
store = file.Storage('../../data/GoogleSheets/storage.json')
creds = store.get()
if(not creds or creds.invalid):
    flow = client.flow_from_clientsecrets(client_secret_file, SCOPES)
    creds = tools.run_flow(flow, store)
# from googleapiclient import discovery
# import pprint
# import json
# spreadsheet_id = '1897f7cqzDOraJ9n8eTv2Za1XmqfKVyF7ScIeAez430k'
# sheet_id = 101650706
# sheet_range = 'B2:V2'
# credentials = [json.loads(l.strip()) for l in open('../../data/GoogleSheets/client_secret.json')][0]
# service = discovery.build('sheets', 'v4', credentials=None)
# request = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=sheet_range)
# response = request.execute()
# pprint(response)

usage: ipykernel_launcher.py [--auth_host_name AUTH_HOST_NAME]
                             [--noauth_local_webserver]
                             [--auth_host_port [AUTH_HOST_PORT [AUTH_HOST_PORT ...]]]
                             [--logging_level {DEBUG,INFO,WARNING,ERROR,CRITICAL}]
ipykernel_launcher.py: error: unrecognized arguments: -f /nethome/istewart6/.local/share/jupyter/runtime/kernel-0e9a2a38-1a87-4938-bbcf-6b38f45d8509.json


SystemExit: 2

In [68]:
from apiclient import discovery
from httplib2 import Http
from oauth2client import file, client, tools
from oauth2client.file import Storage
SCOPES = 'https://www.googleapis.com/auth/spreadsheets.readonly'
CLIENT_SECRET_FILE = 'client_secret.json'
APPLICATION_NAME = 'Google Sheets API Python Quickstart'

def get_credentials():
    """Gets valid user credentials from storage.

    If nothing has been stored, or if the stored credentials are invalid,
    the OAuth2 flow is completed to obtain the new credentials.

    Returns:
        Credentials, the obtained credential.
    """
    credential_path = '../../data/GoogleSheets/client_secret.json'

#     store = Storage(credential_path)
#     credentials = store.get()
#     if not credentials or credentials.invalid:
    flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
    flow.user_agent = APPLICATION_NAME
    if flags:
        credentials = tools.run_flow(flow, store, flags)
    else: # Needed only for compatibility with Python 2.6
        credentials = tools.run(flow, store)
    print('Storing credentials to ' + credential_path)
    return credentials


"""Shows basic usage of the Sheets API.

Creates a Sheets API service object and prints the names and majors of
students in a sample spreadsheet:
https://docs.google.com/spreadsheets/d/1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms/edit
"""
credentials = get_credentials()
http = credentials.authorize(httplib2.Http())
discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?'
                'version=v4')
service = discovery.build('sheets', 'v4', http=http,
                          discoveryServiceUrl=discoveryUrl)

spreadsheetId = '1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms'
rangeName = 'Class Data!A2:E'
result = service.spreadsheets().values().get(
    spreadsheetId=spreadsheetId, range=rangeName).execute()
values = result.get('values', [])

if not values:
    print('No data found.')
else:
    print('Name, Major:')
    for row in values:
        # Print columns A and E, which correspond to indices 0 and 4.
        print('%s, %s' % (row[0], row[4]))

NameError: global name 'flags' is not defined

In [65]:
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials

json_key = json.load(open('../../data/GoogleSheets/service_client_creds.json')) # json credentials you downloaded earlier
scope = ['https://spreadsheets.google.com/feeds']

credentials = ServiceAccountCredentials(json_key['client_email'], json_key['private_key'].encode(), scope) # get email and key from creds

file = gspread.authorize(credentials) # authenticate with Google
sheet = file.open("MUO_Python_Sheet").sheet1 # open sheet

AttributeError: 'str' object has no attribute 'sign'

We gave up and used [this](https://developers.google.com/sheets/api/quickstart/python) boilerplate [here](get_data_from_google_sheets.py).

In [166]:
import codecs
annotator_1_data = list(codecs.open('../../data/facebook-maria/all_group_sample_statuses_annotated_topo_annotator_1.txt', 'r', encoding='utf-8'))
# fix last data point
annotator_1_data[-2] = annotator_1_data[-1].split('//')[0]
annotator_1_data[-1] = annotator_1_data[-1].split('//')[1]
print('\n'.join(annotator_1_data))

barrio ingenio en yabucoa (5184195284); humacao(4468396); candeloro arriba(=UN)

Quintas De Guasimas (=UN); Bda Marin(Node: Barriada Marín (5184347328)

739 carr 104 , Mayaquez PR (22196127);  Mayaquez PR (1944587067)

San Cristóbal Apartments (=UN); Tres Caminos (5184329391); Tres Camino (22194786); Barrancas (4468210); la gallera (VAGUE)

San Juan(4468264); Carolina (4468017);  Bayamón (4468234); Camuy(4467841); Aguada (4468470); Mayagüez (4467818); Añasco (4468537); Carolina (1902275067); aeropuerto Rafael Hernández de Aguadilla (N 18° 29' 43''	W 67° 8' 5'); Antigua Base Ramey en Aguadilla (506154128)

NARANJITO el sector La Sabana(5184324841)

Hogar Ramonita (=UN); Calle Jade en Urbanización La Plata , Cayey(22142874)

Barrio Nuevo (N 18° 17' 23''	W 66° 13' 19'); escuela Don Manolo de Naranjito (N 18° 17' 33''	W 66° 13' 15''); NY(61320)

Palmas Bajas, Guayama (540517197)

urbanización villamar en la vuelta de la culebra (=UN); Guayama (4467974)

Las Parcelas Carmen (=UN)

Carr 4485

In [172]:
id_matcher = re.compile('\(([^\)]+)\)')
id_num_matcher = re.compile('\(([0-9]+)\)')
lat_lon_matcher = re.compile(u'([0-9]+)° ([0-9]+)\' ([0-9]+)')
time_to_deg = lambda x: float(x[0]) + float(x[1]) / 60 + float(x[2]) / 3600
def extract_annotator_data(x):
    x_full_names = map(lambda y: y.strip(), x.split(';'))
    x_full_names = filter(lambda y: y.strip() != '', x_full_names)
    # extract data
    x_data = []
    for x_full in x_full_names:
        x_name = x_full.split('(')[0].strip()
        x_id = 0
        x_lat = 0
        x_lon = 0
        if(len(lat_lon_matcher.findall(x_full)) > 0):
            # convert to decimal
            x_lat_group, x_lon_group = lat_lon_matcher.findall(x_full)
            x_lat = time_to_deg(x_lat_group)
            x_lon = -time_to_deg(x_lon_group) # always convert to negative because we know it's West
        elif(len(id_num_matcher.findall(x_full)) > 0):
            x_id = int(id_num_matcher.findall(x_full)[0])
        else: # UN, VAGUE
            x_id = id_matcher.findall(x_full)[0]
            x_id = x_id.replace('=', '')
        x_data.append([x_name, x_id, x_lat, x_lon])
    return x_data
annotator_1_extracted = map(extract_annotator_data, annotator_1_data)

In [173]:
annotator_1_extracted

[[[u'barrio ingenio en yabucoa', 5184195284, 0, 0],
  [u'humacao', 4468396, 0, 0],
  [u'candeloro arriba', u'UN', 0, 0]],
 [[u'Quintas De Guasimas', u'UN', 0, 0], [u'Bda Marin', 5184347328, 0, 0]],
 [[u'739 carr 104 , Mayaquez PR', 22196127, 0, 0],
  [u'Mayaquez PR', 1944587067, 0, 0]],
 [[u'San Crist\xf3bal Apartments', u'UN', 0, 0],
  [u'Tres Caminos', 5184329391, 0, 0],
  [u'Tres Camino', 22194786, 0, 0],
  [u'Barrancas', 4468210, 0, 0],
  [u'la gallera', u'VAGUE', 0, 0]],
 [[u'San Juan', 4468264, 0, 0],
  [u'Carolina', 4468017, 0, 0],
  [u'Bayam\xf3n', 4468234, 0, 0],
  [u'Camuy', 4467841, 0, 0],
  [u'Aguada', 4468470, 0, 0],
  [u'Mayag\xfcez', 4467818, 0, 0],
  [u'A\xf1asco', 4468537, 0, 0],
  [u'Carolina', 1902275067, 0, 0],
  [u'aeropuerto Rafael Hern\xe1ndez de Aguadilla',
   0,
   18.49527777777778,
   -67.13472222222222],
  [u'Antigua Base Ramey en Aguadilla', 506154128, 0, 0]],
 [[u'NARANJITO el sector La Sabana', 5184324841, 0, 0]],
 [[u'Hogar Ramonita', u'UN', 0, 0],
  [u'

## Compare!!

In [176]:
from __future__ import division
from geopy.distance import great_circle
from itertools import izip
# get total agreement in terms of (1) spans, (2) IDs, (3) lat/lon where able
def get_agreement_stats(x_1, x_2):
    """
    Compare one post against another
    and compute agreement.
    
    Parameters:
    -----------
    x_1 : str
    x_2 : str
    
    Returns:
    --------
    span_diff : int
    span_overlap : float
    id_overlap : float
    coord_dist : float
    """
    x_1_names, x_1_ids, x_1_lats, x_1_lons = zip(*x_1)
    x_2_names, x_2_ids, x_2_lats, x_2_lons = zip(*x_2)
    # compute span, id overlap as basic proportions
    span_diff = abs(len(x_1_names) - len(x_2_names))
    span_overlap = len(set(x_1_names) & set(x_2_names)) / max(len(x_1_names), len(x_2_names))
    if(span_overlap == 0):
        print('0 span overlap between names %s and %s'%(x_1_names, x_2_names))
    id_overlap = len(set(x_1_ids) & set(x_2_ids)) / max(len(x_1_ids), len(x_2_ids))
    if(id_overlap == 0):
        print('0 ID overlap between names %s and %s'%(x_1_names, x_2_names))
    # coordinate overlap is harder! make sure that we're only comparing legit coordinates
    coord_pairs = []
    coord_dist = 0.
    for i, x_1_name in enumerate(x_1_names):
        x_1_name = unicode(x_1_name.decode('utf-8'))
        for j, x_2_name in enumerate(x_2_names):
            x_2_name = unicode(x_2_name)
            if(x_1_name == x_2_name or x_1_name in x_2_name or x_2_name in x_1_name):
                if(x_1_lats[i] != 0 and x_1_lons[i] != 0 and 
                   x_2_lats[j] != 0 and x_2_lons[j] != 0):
                    coord_dist_i_j = great_circle([x_1_lats[i], x_1_lons[i]], 
                                              [x_2_lats[i], x_2_lons[j]]).miles
                    coord_dist += coord_dist_i_j
    return span_diff, span_overlap, id_overlap, coord_dist

In [177]:
sample_label_list
for i, (sample_labels, annotator_1_labels) in enumerate(izip(sample_label_list, annotator_1_extracted)):
#     print(sample_labels)
#     print(annotator_1_labels)
    span_diff_1, span_overlap_1, id_overlap_1, coord_dist_1 = get_agreement_stats(sample_labels, annotator_1_labels)
    print('post %d had span diff %d, span overlap %.3E, id overlap %.3E, coord dist %.3E'%
          (i, span_diff_1, span_overlap_1, id_overlap_1, coord_dist_1))

post 0 had span diff 1, span overlap 5.000E-01, id overlap 2.500E-01, coord dist 0.000E+00
post 1 had span diff 0, span overlap 1.000E+00, id overlap 1.000E+00, coord dist 0.000E+00
0 span overlap between names ('carr 104', 'Mayaquez') and (u'739 carr 104 , Mayaquez PR', u'Mayaquez PR')
post 2 had span diff 0, span overlap 0.000E+00, id overlap 5.000E-01, coord dist 0.000E+00
post 3 had span diff 0, span overlap 8.000E-01, id overlap 6.000E-01, coord dist 0.000E+00
post 4 had span diff 2, span overlap 3.333E-01, id overlap 6.667E-01, coord dist 2.053E-02
0 span overlap between names ('NARANJITO', 'La Sabana') and (u'NARANJITO el sector La Sabana',)
post 5 had span diff 1, span overlap 0.000E+00, id overlap 5.000E-01, coord dist 0.000E+00
0 span overlap between names ('Calle Jade', 'Urbanizaci\xc3\xb3n La Plata', 'Cayey') and (u'Hogar Ramonita', u'Calle Jade en Urbanizaci\xf3n La Plata , Cayey')
post 6 had span diff 1, span overlap 0.000E+00, id overlap 6.667E-01, coord dist 0.000E+00
0

Slight disagreements in toponym combination:

- `Palmas Bajas` + `Guayama` =/= `Palmas Bajas, Guayama`

Non-trivial ID mistakes due to discrepancies in county versus town:

- `Barrio Nuevo` (county) vs. `Barrio Nuevo` (town)

Hopefully these will get ironed out as the annotators improve and get more standardized.

## Annotator 2 comparison
Once we hire Annotator 2...do the same thing.